In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from sample.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number

import ast
from sample.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from sample.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from sample.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from sample.pipeline.functions.experiment_class_surf import Experiment
from sample.paths.directory import run_parallel, find_state, directory_scratch, directory_project, path_code

from sample.notebooks.analysis.data_info import *
import matplotlib.patches as mpatches
from statsmodels.stats import weightstats as stests
from mpl_toolkits.mplot3d import Axes3D
from scipy.ndimage import gaussian_filter
from scipy.ndimage import uniform_filter1d
from scipy import stats

In [2]:
window=800
infos = pickle.load(open(f'{path_code}/MscThesis/Results/straight_bait_{window}.pick', "rb"))

In [3]:
set(infos['treatment'].values)

{'25', '25*', 'baits'}

In [4]:
def get_angle(xa,ya,xb,yb):
    dot_product = (xa*xb+ya*yb)/np.sqrt((xa**2+ya**2)*(xb**2+yb**2))
    angle = (np.arccos(dot_product) / (2 * np.pi) * 360)*(1-2*((ya * xb - yb * xa) >= 0))
    return(angle)

In [6]:
infos.columns

Index(['plate', 'inst', 'treatment', 'angle', 'curvature', 'density', 'growth',
       'speed', 'straightness', 'dist_P', 'dist_N', 'angle_to_P', 'angle_to_N',
       't', 'hyph', 'x', 'y', 'vx', 'vy', 'xinit', 'yinit', 'density5',
       'grad_density_x5', 'grad_density_y5', 'grad_density_norm5', 'density10',
       'grad_density_x10', 'grad_density_y10', 'grad_density_norm10',
       'density20', 'grad_density_x20', 'grad_density_y20',
       'grad_density_norm20'],
      dtype='object')

In [5]:
infos['abs_angle_to_P']=np.abs(infos['angle_to_P'])
infos['abs_curvature']=np.abs(infos['curvature'])
infos['curvature_scaled']=infos['curvature']*np.sqrt(infos['growth'])
infos['abs_curvature_sq']=np.abs(infos['curvature'])*np.sqrt(infos['growth'])
infos['inv_dens']=1/infos['density']
infos['side_cross']=infos.apply (lambda row: comments[row['plate']] if row['plate'] in comments.keys() else 'None', axis=1)
infos['x']=infos['x'].astype(float)
infos['vx']=infos['vx'].astype(float)
infos['y']=infos['y'].astype(float)
infos['vy']=infos['vy'].astype(float)
infos['xinit']=infos['xinit'].astype(float)
infos['yinit']=infos['yinit'].astype(float)
blur = 20
infos['v'] = np.sqrt((infos['vx']**2+infos['vy']**2).astype(float))
infos['gd'] = np.sqrt((infos[f'grad_density_x{blur}']**2+infos[f'grad_density_y{blur}']**2).astype(float))
infos['spvgd']=(infos['vx']*infos[f'grad_density_x{blur}']+infos['vy']*infos[f'grad_density_y{blur}'])/(infos['v']*infos['gd'])
infos['angle_vgd']=get_angle(-infos[f'grad_density_x{blur}'],-infos[f'grad_density_y{blur}'],infos['vx'],infos['vy'])
infos['angle_vgd2']=get_angle(-infos[f'grad_density_x{blur}'],-infos[f'grad_density_y{blur}'],infos['xinit'],infos['yinit'])
infos['angle_Pgd']=infos['angle_vgd']+infos['angle_to_P']
infos['angle_Ngd']=infos['angle_vgd']+infos['angle_to_N']

In [222]:
corrected

,plate,inst,treatment,angle,curvature,density,growth,speed,straightness,dist_P,...,grad_density_norm20,abs_angle_to_P,inv_dens,side_cross,v,gd,spvgd,angle_vgd,angle_Pgd,angle_Ngd
81,351,"(35, 0, 15)",25,21.097950,0.015937,2.200000,1323.871715,330.967929,0.980145,53041.478225,...,0.000399,154.107209,0.454545,None,752.224036,0.001916,-0.667219,48.147240,-105.959970,-57.017925
187,351,"(35, 0, 15)",25,14.828640,0.013928,0.723188,1064.686932,267.285422,0.979827,38423.583893,...,0.000569,144.793003,1.382766,None,604.758630,0.000743,-0.999042,2.508810,-142.284194,87.625502
329,373,"(29, 0, 20)",25,-7.685527,-0.006196,1.979710,1240.499275,310.124819,0.993162,15995.456306,...,0.000173,117.259814,0.505124,None,714.212853,0.001090,0.041641,-92.386568,-209.646383,-58.430462
331,373,"(29, 0, 20)",25,22.236820,0.021098,3.568841,1053.996702,264.601683,0.979901,16388.520896,...,0.000169,130.733018,0.280203,None,598.731993,0.001196,0.505484,-120.363461,-251.096479,-96.337733
397,296,"(9, 0, 11)",25,-16.271150,-0.035884,4.997101,453.443399,344.387391,0.953821,52451.052681,...,0.000216,147.583627,0.200116,None,250.726943,0.003488,-0.810704,-35.835243,111.748383,-181.458008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8169,424,"(13, 116, 131)",baits,-6.179256,-0.005346,1.505797,1155.925879,288.981470,0.990405,52748.786536,...,0.000275,59.898287,0.664100,left,663.672359,0.001454,-0.997131,-4.341156,-64.239443,-152.691377
8170,424,"(13, 116, 131)",baits,2.106778,0.001921,1.225362,1096.650923,274.162731,0.988109,52195.683631,...,0.000332,57.831122,0.816085,left,628.179911,0.001531,-0.958835,-16.496905,-74.328027,-164.831425
8171,424,"(13, 116, 131)",baits,-1.005086,-0.000939,1.239130,1070.490054,267.622514,0.983353,51477.805612,...,0.000358,55.790694,0.807018,left,610.243394,0.001166,-0.965182,-15.163819,-70.954513,-163.719598
8203,424,"(13, 116, 131)",baits,8.569142,0.008203,0.688406,1044.650447,261.162612,0.995721,54646.811463,...,0.000431,112.666397,1.452632,left,603.003317,0.001650,-0.998372,3.270099,-109.396298,174.230080


In [6]:
corrected = infos.loc[(infos["straightness"] <= 1)& (infos["speed"] >=25)& (infos["speed"] <400)&(infos["straightness"] > 0.95)&(infos["density"]>0)]
baits = corrected.loc[corrected['treatment']=='baits']
no_baits = corrected .loc[corrected ['treatment']=='25']
no_baits_new = corrected .loc[corrected ['treatment']=='25*']

In [8]:
right_crossing = [plate for plate in comments.keys() if comments[plate]=='right']

In [18]:
plt.close('all')
plate_select = corrected.loc[corrected ['plate']==94]
# plate_select = corrected.loc[corrected ['plate'].isin(right_crossing)]

abcisse = 'angle_to_N'
ordinate = 'spvgd'
tab = no_baits
baits_sort = tab.sort_values(abcisse)
N=600
moving_av = baits_sort.rolling(N,min_periods=N//2).mean()
moving_std = baits_sort.rolling(N,min_periods=N//2).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('angle to N (°)')
ax.set_ylabel('normalised scalar product speed.density gradient')
# ax.set_xlim(-0.5,0.5)
# ax.set_ylim(-10,10)
# ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
for plate in set(tab['plate'].values):
    select = tab.loc[tab['plate']==plate]
    ax.scatter(select[abcisse],select[ordinate],label=plate,alpha=0.3)
ax.plot(moving_av[abcisse],moving_av[ordinate],color='green',label = 'moving average')
ax.plot(moving_av[abcisse],(moving_av[ordinate]+moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')
ax.plot(moving_av[abcisse],(moving_av[ordinate]-moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')
# ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
from scipy.interpolate import interp1d
x = moving_av["inv_dens"]
y = moving_av["speed"]
f = interp1d(x, y,copy=True,fill_value=(150,240),bounds_error=False)

In [8]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
bplot1 = corrected.boxplot(column = ['gd'],by="treatment",figsize =(9,8),ax =ax,patch_artist=True, showfliers=False, notch=True,showmeans = True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
tab

,plate,inst,treatment,angle,curvature,density,growth,speed,straightness,dist_P,...,abs_angle_to_P,inv_dens,side_cross,v,gd,spvgd,angle_vgd,angle_vgd2,angle_Pgd,angle_Ngd
0,69,"(40, 153, 190)",25*,-24.883829,-0.010112,0.962319,2460.850743,307.606343,0.979885,39704.557927,...,6.580413,1.039157,None,1397.884831,0.001005,-0.949730,-18.244346,-7.096006,-24.824759,139.640999
1,69,"(40, 153, 190)",25*,38.655532,0.045338,0.869565,852.600387,213.150097,0.968758,37310.115919,...,4.524071,1.150000,None,478.819381,0.001209,-0.986953,-9.265307,-9.184084,-4.741236,163.237323
2,69,"(40, 153, 190)",25*,25.516456,0.043642,1.508696,584.681289,146.170322,0.972825,46687.278602,...,125.000209,0.662824,None,329.734742,0.001330,-0.930811,21.438431,10.614160,-103.561778,20.261635
6,69,"(40, 153, 190)",25*,-10.407711,-0.011552,2.052899,900.982105,225.245526,0.996245,47844.264278,...,73.126647,0.487116,None,520.347000,0.001644,-0.857094,-31.008209,-28.915640,-104.134856,13.155560
7,69,"(40, 153, 190)",25*,-16.336043,-0.022268,1.194928,733.595192,174.665522,0.988511,49581.350710,...,50.872094,0.836871,None,420.386727,0.000845,-0.825177,-34.393538,-25.487273,-85.265631,29.078085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13035,424,"(13, 116, 131)",baits,7.281669,0.012852,0.903623,566.586163,141.646541,0.968746,47451.023860,...,39.741216,1.106656,left,318.190195,0.001332,-0.103831,-84.040167,-85.238858,-44.298951,-146.722969
13036,424,"(13, 116, 131)",baits,-20.905807,-0.030679,2.035507,681.432714,170.358178,0.984990,46972.393814,...,35.315072,0.491278,left,389.104099,0.002163,-0.846596,-32.156702,-25.034267,3.158370,-101.492343
13038,424,"(13, 116, 131)",baits,24.176796,0.050994,1.960870,474.114541,118.528635,0.992319,45906.669249,...,134.782141,0.509978,left,272.737969,0.001703,-0.396878,66.616828,50.280858,-68.165313,178.926660
13039,424,"(13, 116, 131)",baits,16.548582,0.029624,2.907971,558.616681,139.654170,0.981037,46240.456212,...,153.106695,0.343882,left,317.694822,0.001560,-0.565473,55.564825,44.458630,-97.541870,152.336760


In [39]:
np.mean(tab['angle_vgd2']),np.std(tab['angle_vgd2'])/np.sqrt(len(tab))

(19.21577680475666, 2.0123061891325618)

In [7]:
plt.close('all')
plate_select = corrected.loc[corrected ['plate']==436]
abcisse = 'spvgd'
ordinate = 'speed'
tab = no_baits_new
baits_sort = tab.sort_values(abcisse)
N=600
moving_av = baits_sort.rolling(N,min_periods=N//2).mean()
moving_std = baits_sort.rolling(N,min_periods=N//2).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('normalised scalar product speed.density gradient')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
# ax.set_xlim(-0.001,0.001)
# ax.set_ylim(-150,150)
# ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
for plate in set(tab['plate'].values):
    select = tab.loc[tab['plate']==plate]
    ax.scatter(select[abcisse],select[ordinate],label=plate,alpha=0.3)
ax.plot(moving_av[abcisse],moving_av[ordinate],color='green',label = 'moving average')
ax.plot(moving_av[abcisse],(moving_av[ordinate]+moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')
ax.plot(moving_av[abcisse],(moving_av[ordinate]-moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:1377: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x[:, None]
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/axes/_base.py:237: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x = x[:, np.newaxis]
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/axes/_base.py:239: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  y = y[:, np.newaxis]


In [112]:
def get_density_map(plate,t,compress,blur,x,y,fz):
    densities=np.zeros((30000//compress,60000//compress),dtype=np.float)
    select = corrected.loc[(corrected['plate']==plate) & (corrected['t']==t)]
    for index,row in select.iterrows():
        xx = int(row[x])//compress
        yy = int(row[y])//compress
        densities[xx,yy]+=fz(row)
    density_filtered = gaussian_filter(densities,blur)
    return(density_filtered)

    

In [113]:
ts = set(corrected.loc[(corrected['plate']==419)]['t'])
ts = list(ts)
ts.sort()

In [229]:
compress = 100
blur = 20
densities=np.zeros((30000//compress,60000//compress),dtype=np.float)
count=np.zeros((30000//compress,60000//compress),dtype=np.float)

select = corrected
for index,row in select.iterrows():
    xx = int(row['x'])//compress
    yy = int(row['y'])//compress
    densities[xx,yy]+=row['angle_vgd']
    count[xx,yy]+=1
density_filtered = gaussian_filter(densities/(count+(count==0).astype(float)),blur)

In [115]:
densities = [get_density_map(419,t,100,5,'x','y',lambda row : row['speed']) for t in ts]

In [230]:
plt.close('all')
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
ax.imshow(density_filtered)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [117]:
imageio.mimsave(f'movie.gif', densities,duration = 1)

Lossy conversion from float64 to uint8. Range [0.0, 1.72143677708428]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 3.957171889756781]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 3.740454175547317]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 3.322802597473528]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 2.498569646579737]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 3.520259599237698]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 2.4290524530575817]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. R